In [ ]:
import os
import sys
import cv2
import math

path_to_video = './monkey.avi'
outputframes = './outputframes/'

k = 4 # any random k such that the following is odd
grid_block = ((2 * k) + 1) # hence block is odd


Helper Functions

In [ ]:
''' Code from Lab 4 '''
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def check_bounds(dimension):
    if dimension[0] < 0 or dimension[0] >= dimension[1]: return True

In [ ]:
def arrowdraw(img, x1, y1, x2, y2):
    x1 = x1 * grid_block
    y1 = y1 * grid_block
    radians = math.atan2(x1-x2, y2-y1)
    x11 = 0
    y11 = 0
    x12 = -10
    y12 = -10

    u11 = 0
    v11 = 0
    u12 = 10
    v12 = -10
    
    x11_ = x11*math.cos(radians) - y11*math.sin(radians) + x2
    y11_ = x11*math.sin(radians) + y11*math.cos(radians) + y2

    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2
    
    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2

    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2
    img = cv2.arrowedLine(img, (x1, y1), (x2, y2), (255, 255, 255), 2)
    img = cv2.arrowedLine(img, (int(x11_), int(y11_)), (int(x12_), int(y12_)), 
    (255, 255, 255), 2)
    img = cv2.arrowedLine(img, (int(u11_), int(v11_)), (int(u12_), int(v12_)), 
    (255, 255, 255), 2)
    
    return img

1. Blockmatching Algorithm

In [ ]:
def blockmatch(Fi ,Fj):

    ''' Blockmatching Algorithm based on the Assignment specification with influence
    from the following sources:

    1. Altman, E. A., & Zakharenko, E. I. (2018, July). 
        A new algorithm of fast SSD calculation for motion estimation. 
        In Journal of Physics: Conference Series (Vol. 1050, No. 1, p. 012002).
        IOP Publishing.

    2. Wikipedia: Block-matching Algorithm
        https://en.wikipedia.org/wiki/Block-matching_algorithm
    
    3. Massanes, F., Cadennes, M., & Brankov, J. G. (2011). Compute-unified device 
        architecture implementation of a block-matching algorithm for multiple 
        graphical processing unit cards. Journal of electronic imaging, 20(3), 033004.
    
    4. Mathworks: Block Matching
        https://www.mathworks.com/help/vision/ref/blockmatching.html

    Args:- 
        Fi: Current Frame
        Fj: Next Frame
    '''

    # Get our height and width restrained by our grid_block of k=4
    Fx = int(frame_height // grid_block)
    Fy = int(frame_width // grid_block)
    block_radius = int((grid_block - 2)) - 1 

    # Random range chosen to reduce noise
    Tmin = 140
    Tmax = 155
    
    # For each grid block B(x,y) or source block in current frame
    for y1 in range(Fx):
        for x1 in range(Fy):

            # Store nearest neighbours without duplicates
            displacement_vectors = set() 

            # Exhaustive method, moves search area one pixel at a time (reference 4)
            Bi = Fi[ y1 * grid_block: y1 * grid_block + grid_block, 
                     x1 * grid_block: x1 * grid_block + grid_block,
                     :]

            # For next block, restrict our displacement_vectors to a regular grid (reference 2/3)
            for y2 in range(y1-block_radius,y1+block_radius):
                    if check_bounds((y2, Fx)): continue
                    for x2 in range(x1-block_radius,x1+block_radius):
                        if check_bounds((x2, Fy)): continue

                        #Bj = [bx', by', bc']
                        Bj = Fj[y2 * grid_block: y2 * grid_block + grid_block, 
                                x2 * grid_block: x2 * grid_block + grid_block,
                                :]
                                
                        ssd = math.sqrt(((Bi - Bj) ** 2).sum())
                        displacement_vectors.add((ssd, x2, y2))


            # Choose block with lowest error.
            vectors = min(displacement_vectors)
            x2 = vectors[1] * grid_block           
            y2 = vectors[2] * grid_block           
            #Neglect noisy displacmenet vectors whose SSD ∉ (tmin, tmax)
            if Tmin < vectors[0] and vectors[0] < Tmax: arrowdraw(Fj, x1, y1, x2, y2)
    
    cv2.imwrite(outputframes + 'frame%d.tif' %count, Fj)

2. Extract Frames from a Video

In [ ]:
frame_save_path = './frames/'

In [ ]:
cap = cv2.VideoCapture(path_to_video)
create_dir_if_not_exists(frame_save_path) # Or you can create it manually

if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)

# time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_counter = 0                                             # FRAME_COUNTER
while(1):
    return_flag, frame = cap.read()
    if not return_flag:
        print('Video Reach End')
        break
    # Main Content - Start
    # cv2.imshow('VideoWindowTitle-Quadrangle', frame)
    cv2.imwrite(frame_save_path + 'frame%d.tif' % frame_counter, frame)
    frame_counter += 1
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
    # Main Content - End
cap.release()

3. Perform Blockmatching for any number of frames 

In [ ]:
path_to_output_video = './monkey_see_monkey_do.mov' # .mov because Mac

In [ ]:
count = 0
create_dir_if_not_exists(outputframes)
frames = 500 # Number of frames to match

while count < frames:
    
    frame1 = cv2.imread(frame_save_path + 'frame%d.tif' %count)
    frame2 = cv2.imread(frame_save_path + 'frame%d.tif' %(count+1))

    if frame1 is None or frame2 is None: break

    blockmatch(frame1, frame2)
    count += 1

if count == frames: print('Finished!. {frames} frames matched successfully!'.format(frames = frames))
else: print('Incomplete. Only {count} frames matched'.format(count = count))


4. Convert the composited frame sequence into video

In [ ]:
out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (int(frame_width), int(frame_height)))
frame_counter = 0

while(1):
    img = cv2.imread(outputframes + 'frame%d.tif' % frame_counter)
    if img is None:
        print('No more frames to be loaded')
        break;
    out.write(img)
    frame_counter += 1
out.release()
cv2.destroyAllWindows()
